In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [3]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [5]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""

    return prompt

In [7]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [8]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also ne

In [10]:
dash_line = '-'.join('' for x in range(100))

with out any things

In [11]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


question1:

max_new_token=10

In [12]:
inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=10
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system.



 **

 پاسخ سوال 1 - پس از تغییر مقدار مکس نیو توکن و تبدیل آن به 10 حداکثر مقدار توکن تولیدی خروجی مدل از 50 توکن که قبلا قرار داده بودیم به 10 توکن کاهش میابد


**

question 2  &  3:



temperature=1,1.5,2,0.7,0.003,0.5



In [16]:
temperatures = [0.03,0.2,0.5,0.7, 1,1, 1.5, 2.0]

print(f"{'Temperature'}:                         | {'Generated Summary'}")
print(dash_line)

for temp in temperatures:
    output_ids = model.generate(
        tokenizer(one_shot_prompt, return_tensors='pt')["input_ids"],
        max_new_tokens=50,
        do_sample=True,
        temperature=temp
    )[0]
    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    print(f"{temp}:                                  | {summary}")

Temperature:                         | Generated Summary
---------------------------------------------------------------------------------------------------
0.03:                                  | #Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.
0.2:                                  | #Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.
0.5:                                  | #Person1 recommends upgrading their system and hardware.
0.7:                                  | A website offers a couple of suggestions for upgrades.
1:                                  | #Person1 is considering to upgrade his/her system, but is not sure what's the best way to accomplish it.
1:                                  | #Person1 explains that he needs to upgrade his computer, plus his hardware. He also explains how he would improve his pro

In [17]:
temperatures = [0.8,0.8,0.9,0.9,0.9,1,1,1,1,1.1,1.1,1.2]

print(f"{'Temperature'}:                         | {'Generated Summary'}")
print(dash_line)

for temp in temperatures:
    output_ids = model.generate(
        tokenizer(one_shot_prompt, return_tensors='pt')["input_ids"],
        max_new_tokens=50,
        do_sample=True,
        temperature=temp
    )[0]
    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    print(f"{temp}:                                  | {summary}")

Temperature:                         | Generated Summary
---------------------------------------------------------------------------------------------------
0.8:                                  | #Person1 wants to sell their software software. He wants to add a painting program and computer hardware to it.
0.8:                                  | Adding a painting program to your software could make your software more useful.
0.9:                                  | They want to upgrade the system, and ask about hardware.
0.9:                                  | People are considering upgrading their systems, but are not sure what they need.
0.9:                                  | There are several suggestions to upgrade your systems.
1:                                  | #Person2 suggests upgrading his computer. He's not sure what to do and how to do it.
1:                                  | #Person1 proposes to upgrade his computer equipment and his computer system. #Person2 offers a p

پاسخ پرسش های 2 و 3 :   هر چه دما به 1 نزدیک تر میشد وضعیت خروجی بهتر بود و هر چه از 1 کمتر بود خلاصه خلاقیت کمتری داشت ولی هر چه از 1 بیشتر میشد ضمن خلاقیت بیشتر متن با فاصله بیشتر از 1 کلا بی ربط تر از متن اصلی میشد

با توجه به بهبود وضعیت در حالت دمای 1 سعی کردم جند بار 1 را روی مدل تست کنیم که هر بار مقدار خروجی متفاوتی داد جون هر بار احتمال برداشتن خروجی ها دوباره متفاوت میشد اما نهایتا تعداد دفعات ازمایش خیلی تاثیری در خروجی نداشت